# ParFuMor (version YAML & Objets)

In [10]:
# -*- coding: utf8 -*-
from os.path import expanduser
import yaml, itertools
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [11]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
numerosKalabas=[1,2,3,4,5]
nomKalabas=[repertoire+"16-K%d/"%num for num in numerosKalabas]
print nomKalabas

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

['/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/', '/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K2/', '/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/', '/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K4/', '/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K5/']


In [12]:
for serie in nomKalabas:
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        stems=yaml.load(stream)
        PFM.stems=stems
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)

    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/
VER : Temps Pers
NOM : Genre Nombre
DET : Genre Nombre
ADJ : Genre Nombre
head stems
head stems,VER
head stems,NOM
head stems,NOM,M
head stems,NOM,F
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K2/
VER : Temps Pers Genre
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
ADJ : Genre Nombre
head stems
head stems,VER
head stems,NOM
head stems,NOM,F
head stems,NOM,M
head stems,NOM,N
head stems,DET
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/
VER : Trans Temps Pers Genre Nombre
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
ADJ : Genre Nombre
head stems
head stems,VER
head stems,VER,V1
head stems,VER,V2
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B
head stems,NOM,D
head stems,DET
head stems,ADJ
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K4/
VER : Trans Temps Pers Genre Nombre
NOM : Ge